In [3]:
import os
import numpy as np
import yaml
from copy import deepcopy
from ultralytics.engine.results import Boxes, Results
from ultralytics.data.utils import check_det_dataset, img2label_paths
from PIL import Image
import numpy as np
import cv2
import torch


In [4]:
def map_yaml_values(src_yaml_path, tgt_yaml_path, mapped_label):
    # Bước 1: Load dữ liệu từ tệp YAML nguồn
    with open(src_yaml_path, 'r') as file:
        data = yaml.safe_load(file)

    # Bước 2: Kiểm tra và ánh xạ các giá trị trong 'names'
    if 'names' not in data:
        raise KeyError("Tệp YAML không chứa khóa 'names'.")

    updated_names = {}
    for key, value in data['names'].items():
        key = int(key)  # Đảm bảo khóa là số nguyên
        if key in mapped_label:
            new_key, new_value = mapped_label[key]
            updated_names[new_key] = new_value

    # Bước 3: Thay thế giá trị của 'names' bằng danh sách đã cập nhật
    data['names'] = updated_names

    os.makedirs(os.path.dirname(tgt_yaml_path), exist_ok=True)

    # Bước 4: Ghi dữ liệu mới vào tệp YAML đích
    with open(tgt_yaml_path, 'w') as file:
        yaml.dump(data, file, default_flow_style=False, allow_unicode=True)

    print(f"Dữ liệu đã được ánh xạ và lưu tại: {tgt_yaml_path}")


def convert_label(src_label_folder, tgt_label_folder, map_label):
    os.makedirs(tgt_label_folder, exist_ok=True)
    # Duyệt qua từng file trong folder
    for filename in os.listdir(src_label_folder):
        if filename.endswith(".txt"):
            file_path = os.path.join(src_label_folder, filename)
            
            # Đọc nội dung của file label
            with open(file_path, "r") as f:
                lines = f.readlines()

            # Cập nhật class_id trong mỗi dòng
            updated_lines = []
            for line in lines:
                components = line.strip().split()
                class_id = int(components[0])
                if class_id in map_label:
                    # Thay đổi class_id theo map mới
                    components[0] = str(map_label[class_id][0])
                    updated_lines.append(" ".join(components) + "\n")

            # Ghi đè lại file với class mới
            with open(os.path.join(tgt_label_folder, filename), "w") as f:
                f.writelines(updated_lines)

    print("Cập nhật class_id hoàn tất.")

In [ ]:
# mapped_label = {
#     0: (1, 'TA'),
#     1: (0, 'TJ'),
#     2: (2, 'S'),
#     3: (2, 'S'),
#     4: (3, 'G'),
#     5: (3, 'G'),
#     6: (4, 'Healthy'),
#     7: (5, 'Others'),
#     8: (5, 'Others'),
#     9: (6, 'Difficult'),
# }

# mapped_label = {
#     0: (1, 'TA'),
#     1: (0, 'TJ'),
#     2: (2, 'S'),
#     3: (2, 'S'),
#     4: (3, 'G'),
#     5: (3, 'G'),
#     6: (4, 'unparasitized'),
#     7: (4, 'unparasitized'),
#     8: (4, 'unparasitized'),
# }

mapped_label = {
    0: (1, 'TA'),
    1: (0, 'TJ'),
    2: (2, 'S1'),
    3: (3, 'S2'),
    4: (4, 'G1'),
    5: (5, 'G25'),
    6: (6, 'unparasitized'),
    7: (6, 'unparasitized'),
    8: (6, 'unparasitized'),
    9: (7, 'Difficult'),
}

# mapped_label = {
#     0: (1, 'TA'),
#     1: (0, 'TJ'),
#     2: (2, 'S1'),
#     3: (3, 'S2'),
#     4: (4, 'G1'),
#     5: (5, 'G2-5'),
#     6: (6, 'Healthy'),
#     7: (7, 'Others'),
#     8: (7, 'Others'),
#     9: (8, 'Difficult'),
# }

In [6]:
# map_yaml_values('datasets/malaria_full_class_temp1/data.yaml', 'datasets/pa3_malaria_7_classes_8folds_temp1/data.yaml', mapped_label)
# map_yaml_values('datasets/malaria_full_classes_temp3/data.yaml', 'datasets/malaria_9_classes_8_folds_temp3/data.yaml', mapped_label)
# map_yaml_values('datasets/v2_malaria_full_class/data.yaml', 'datasets/v2_malaria_PA3_7_class_edge_filtered/data.yaml', mapped_label)
map_yaml_values('datasets/v2_malaria_full_class/data.yaml', 'datasets/v2_malaria_PA7_5_class/data.yaml', mapped_label)
# map_yaml_values('datasets/v2_malaria_full_class/data.yaml', 'datasets/v2_malaria_PA8_8_class/data.yaml', mapped_label)

Dữ liệu đã được ánh xạ và lưu tại: datasets/v2_malaria_PA7_5_class/data.yaml


In [7]:
convert_label('datasets/v2_malaria_full_class/train/labels',
              'datasets/v2_malaria_PA7_5_class/train/labels',  
              mapped_label)
convert_label('datasets/v2_malaria_full_class/val/labels',
              'datasets/v2_malaria_PA7_5_class/val/labels',  
              mapped_label)
convert_label('datasets/v2_malaria_full_class/test/labels',
              'datasets/v2_malaria_PA7_5_class/test/labels',  
              mapped_label)

Cập nhật class_id hoàn tất.
Cập nhật class_id hoàn tất.
Cập nhật class_id hoàn tất.


In [11]:
convert_label('datasets/v2_malaria_full_class_edge_filtered/train/labels',
              'datasets/v2_malaria_PA3_7_class_edge_filtered/train/labels',  
              mapped_label)

Cập nhật class_id hoàn tất.


In [12]:
convert_label('datasets/v2_malaria_full_class_edge_filtered/val/labels',
              'datasets/v2_malaria_PA3_7_class_edge_filtered/val/labels',  
              mapped_label)

Cập nhật class_id hoàn tất.


In [13]:
convert_label('datasets/v2_malaria_full_class_edge_filtered/test/labels',
              'datasets/v2_malaria_PA3_7_class_edge_filtered/test/labels',  
              mapped_label)

Cập nhật class_id hoàn tất.


In [6]:

def draw_gt(path, data_yaml_path):
    image = Image.open(path)
    image_np = np.array(image).astype(np.uint8)
    image_np = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)

    data_yaml = check_det_dataset(data_yaml_path)
    
    img_height, img_width = image_np.shape[:2]
    label_path = img2label_paths([path.replace("/", "\\")])[0]
    boxes_data = []
    with open(label_path, 'r') as f:
        # Đọc từng dòng
        for line in f:
            # Tách các giá trị trong dòng (theo khoảng trắng)
            values = line.split()
            
            # Lấy class_id, x_center, y_center, width, height
            class_id = int(values[0])
            x_center = float(values[1])
            y_center = float(values[2])
            width = float(values[3])
            height = float(values[4])

            x_min = int((x_center - width / 2) * img_width)
            y_min = int((y_center - height / 2) * img_height)
            x_max = int((x_center + width / 2) * img_width)
            y_max = int((y_center + height / 2) * img_height)
            box_tensor = torch.tensor([x_min, y_min, x_max, y_max, 1.0, class_id])
            boxes_data.append(box_tensor)

    boxes_data = torch.stack(boxes_data)
    # print(boxes_data.ndim)
    # boxes = Boxes(boxes_data, (img_height, img_width))
    results = Results(image_np, path, data_yaml['names'], boxes=boxes_data)

    # for result in results:
    results.save_crop(save_dir=os.path.join('groundtruth', os.path.basename(path)), file_name="detection")
    results.save(os.path.join('groundtruth', os.path.basename(path), "annotated_image.jpg"), conf=False, line_width=4)
    return



In [8]:

path='datasets/v2_malaria_PA3_7_class_edge_filtered/test/images/068.jpg'
data_yaml_path = 'datasets/v2_malaria_PA3_7_class_edge_filtered/data.yaml'
draw_gt(path, data_yaml_path)